In [1]:
import pandas as pd
from sklearn import preprocessing
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
import numpy as np

from random import sample

In [2]:
fileName = "imdb-AiClass.csv"
df = pd.read_csv(fileName)

In [3]:
df.head()

,title,year,genre,duration,country,director,writer,production_company,actors,avg_vote,metascore,genre 1,genre 2,genre 3
0,Intolerance: Love's Struggle Throughout the Ages,1916,"Drama, History",163,USA,D.W. Griffith,"D.W. Griffith, Anita Loos",Triangle Film Corporation,"Lillian Gish, Mae Marsh, Robert Harron, F.A. T...",78.0,93.0,Drama,History,None
1,Metropolis,1927,"Drama, Sci-Fi",153,Germany,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",83.0,98.0,Drama,Sci-Fi,None
2,The Jazz Singer,1927,"Drama, Music, Musical",88,USA,Alan Crosland,"Samson Raphaelson, Alfred A. Cohn",Warner Bros.,"Al Jolson, May McAvoy, Warner Oland, Eugenie B...",65.0,66.0,Drama,Music,Musical
3,The Circus,1928,"Comedy, Romance",72,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Al Ernest Garcia, Merna Kennedy, Harry Crocker...",82.0,90.0,Comedy,Romance,None
4,Piccadilly,1929,"Drama, Crime",92,UK,Ewald André Dupont,Arnold Bennett,British International Pictures (BIP),"Gilda Gray, Anna May Wong, Jameson Thomas, Cyr...",72.0,89.0,Drama,Crime,None


In [4]:
genres = list(df['genre 1'].unique())
g2 = df['genre 2'].unique()
for g in g2:
    if g not in genres:
        genres.append(g)

g3 = df['genre 3'].unique()
for g in g3:
    if g not in genres:
        genres.append(g)

# print(g1)

In [5]:
def genreDfMaker(dfx,genre):
    dx0 = dfx[dfx['genre 1'] == genre]
    dx1 = dfx[dfx['genre 2'] == genre]
    dx2 = dfx[dfx['genre 3'] == genre]
    
    dx = dx0.append(dx1)
    dx = dx.append(dx2)
    
    return dx

In [6]:
# generate similarityDictionary = {genre:{clusterLabel:[movies]}}
# movies - movietitle-director-year
# genreModel = {genre:classificationModel}
pointsPerLabel = 10
similarityDictionary = {}
genreModel = {}

for g in genres:
    if g == 'None':
        continue
        
    dfg = genreDfMaker(df,g)
    X = dfg[['avg_vote','metascore']]
    
    clusters = int(len(dfg)/pointsPerLabel) + 1
    kmeans = KMeans(n_clusters=clusters, random_state=0).fit(X)
    
    genreModel[g] = kmeans
    similarityDictionary[g] = {}
    
    count = 0
    for i, j in dfg.iterrows():
        centerLabel = kmeans.labels_[count]
        movieTitle = j['title'] + '|' + j['director'] + '|' + str(j['year'])
        
        if centerLabel in similarityDictionary[g]:
            similarityDictionary[g][centerLabel].append(movieTitle)
        else:
            similarityDictionary[g][centerLabel] = [movieTitle]
            
        count += 1

print(similarityDictionary)

{'Drama': {108: ["Intolerance: Love's Struggle Throughout the Ages|D.W. Griffith|1916", 'The Magnificent Ambersons|Orson Welles, Fred Fleck|1942', 'The Manchurian Candidate|John Frankenheimer|1962', 'The Servant|Joseph Losey|1963', 'Days of Heaven|Terrence Malick|1978', 'Amour|Michael Haneke|2012', 'Before Midnight|Richard Linklater|2013', 'Call Me by Your Name|Luca Guadagnino|2017', 'Stagecoach|John Ford|1939', 'The Searchers|John Ford|1956', 'Badlands|Terrence Malick|1973', 'Dunkirk|Christopher Nolan|2017', 'The French Connection|William Friedkin|1971'], 302: ['Metropolis|Fritz Lang|1927', 'All About Eve|Joseph L. Mankiewicz|1950', 'The Army of Shadows|Jean-Pierre Melville|1969', "Pan's Labyrinth|Guillermo del Toro|2006", 'The Treasure of the Sierra Madre|John Huston|1948', 'Rashômon|Akira Kurosawa|1950'], 109: ['The Jazz Singer|Alan Crosland|1927', 'Al di là delle nuvole|Michelangelo Antonioni, Wim Wenders|1995', 'Hermanas|Julia Solomonoff|2005', 'Stake Land|Jim Mickle|2010', 'Byzan

In [7]:
# user= {'title':score(personal)}
# userClusterScore= {genre:{clusterNo:[score(calculated),no of reviews]}}
# users = {unserName:{user,ussecluster}}
def login(users):
    userName = input("Enter User Name: ")
    if userName in users:
        print('Welcome back ',userName)
        user = users[userName][0]
        userClusterScore = users[userName][1]
    else:
        print('Welcome to recomendation engine')
        user = {}
        userClusterScore = {}
        users[userName] = [user,userClusterScore]
    return user, userClusterScore

In [8]:
def makeRecommendations(dfx, userSeenMovies, userGroupScore, similarityDictionary):
    recommendations = []
    
    for genre in userGroupScore:
        glist = userGroupScore[genre]
        
        for cluster in glist:
            score = glist[cluster][0]
            reviews = glist[cluster][1]
            
            # factor of how many samples to take, raised by scale
            factor = 0
            if score < 50:
                break
            
            elif score < 60:
                factor = 0.2
                
            elif score < 75:
                factor = 0.4
            
            elif score < 90:
                factor = 0.8
            
            else:
                factor = 0.9
                
            scale = 1
            if reviews >5 :
                scale = 1.1
            else:
                scale = reviews/5
            
            similarMovies = similarityDictionary[genre][cluster]
            glen = len(similarMovies)
            
            samples = int((factor**scale) * glen)
            if samples > glen:
                samples = glen
            
            similarMovies = sample(similarMovies,samples)
            
            # recommendations = recommendations + similarMovies - seen movies
            for movie in similarMovies:
                if movie not in userSeenMovies:
                    recommendations.append(movie)
                
            
    if len(recommendations) < 50:
        hidf = dfx[dfx['avg_vote'] > 80]

        name = list(hidf['title'])
        director = list(hidf['director'])
        year = list(hidf['year'])
        uTitles = [name[i] + '-' + director[i] + '-' + str(year[i]) for i in range(len(mdf))]
        goodMovies = sample(uTitles, 50-len(recommendations))
        
        recommendations = recommendations + goodMovies
    
    return sample(recommendations,10)

# makeRecommendations(df, user, userClusterScore, similarityDictionary)

In [9]:
# prob of inception given i 
# p(i)incepion = no movies with i/no of movies
def deepSearch(movies,query):
    close = []
    common = ['of','in','and','the','to','with']
    queryWords = query.lower().split()
    
    for movie in movies:
        movielow = movie.lower()
        movieWords = movielow.split()
        mWStarts = [word[:4] for word in movieWords]
        # qx = queryBroken[:]
        
        for movieWord in movieWords:
            if movieWord in common:
                continue
                
        for queryWord in queryWords:
            if queryWord[:4] in mWStarts:
                # print(movie,queryWord)
                close.append(movie)
                break
        
    
    # Incase Too many results only use query with 1 exact match word
    if len(close) < 10:
        return close

    else:
        # matched = {movie:mathcedword}
        matched = {}
        shortList = close[:]
        close2 = []
        
        for movie in shortList:
            movielow = movie.lower()
            movieWords = movielow.split(' ')
            # qx = queryBroken[:]

            for movieWord in movieWords:
                if movieWord in common:
                    continue

            for queryWord in queryWords:
                if queryWord in movieWords:
                    matched[movie] = queryWord
                    close2.append(movie)
                    break
            # print(end movie word loop)
    
    # if still too big matchother parts of query
    if len(close2) < 10:
        if len(close2) == 0:
            return close
        return close2

    else:
        # matched = {movie:mathcedword}
        shortList = close2[:]
        close3 = []
        
        for movie in shortList:
            movielow = movie.lower()
            movieWords = movielow.split(' ')
            mWStarts = [word[:4] for word in movieWords]
        
            for movieWord in movieWords:
                if movieWord in common:
                    continue

            for queryWord in queryWords:
                if queryWord == matched[movie]:
                    continue
                if queryWord[:4] in mWStarts:
                    close3.append(movie)
                    break
    
    if len(close3) == 0:
        return close2
    return close

In [28]:
# search with scoring- too buggy
#         if len(shortenedList) < 8:
#             return shortenedList
    
#     # if there are many similar queries score
#     for movie in shortenedList:
#         movielow = movie.lower()
#         movieWords = movielow.split()
        
#         # get a score backward(query-words) and forward(words-queries) for best results
#         for word in movieWords:
#             fscore = 0
#             bscore = 0
#             if word not in common and word in queryWords:
#                 fscore = 1
#                 close.append(movie)
#                 continue

#             for queryWord in queryWords:
#                 tfscore = 0
#                 queryBroken = list(queryWord)
                
#                 # forward score
#                 wordX = list(word)
#                 for letter in wordX:
#                     if letter in queryBroken:
#                         queryBroken = queryBroken[queryBroken.index(letter)+1:]
#                         tfscore += 1
#                         if len(queryBroken) == 0:
#                             break
                            
#                 tscore = tfscore/len(word)
#                 if tfscore > fscore:
#                     fscore = tscore
                
#                 #backward score
#                 queryBroken = list(queryWord)
#                 for qx in queryBroken:
#                     if qx in wordX:
#                         wordx = wordx[wordx.index(qx)+1:]
#                         bscore += 1
#                         if len(wordx) == 0:
#                             break
                
#                 if fscore > 0.7 and bscore > 0.7:
#                     close.append(movie)
            
#             # end query loop
#         # end word loop
#     #end shortlist


#     return close

In [10]:
def locateMovie(dfx):
    movieName = input('Which movie are you reviewing: ')
    movieName = movieName.title()
    movieDf = dfx[dfx['title']==movieName]
    
    if len(movieDf) == 0:
        closeMovies = deepSearch(list(dfx['title']), movieName)
        
        if len(closeMovies) == 0:
            print('Movie not found')
            return -1
        
        movieDf = dfx[dfx['title'] == closeMovies[0]]
        if len(closeMovies) > 1:
            for i in range(1,len(closeMovies)):
                d1 = dfx[dfx['title'] == closeMovies[i]]
                movieDf = movieDf.append(d1)
        movieDf = movieDf.drop_duplicates()

    
    if(len(movieDf) > 1):
        print('Which of these are you referring to-')
        movieTitles = list(movieDf['title'])
        movieYears = list(movieDf['year'])
        movieGenres = list(movieDf['genre 1'])
        movieDirectors = list(movieDf['director'])
        for i in range (len(movieTitles)):
            print((i+1),'.',movieTitles[i],'|',movieDirectors[i],'|',movieYears[i], '|', movieGenres[i])
        
        n = input()
        if n.isnumeric():
            n = int(n)
        else:
            return
        
        if n==0 or n > len(movieDf):
            return
        
        movieDf = movieDf[movieDf['year']==movieYears[n-1]]
        movieDf = movieDf[movieDf['director']==movieDirectors[n-1]]
    
    return movieDf
 
# locateMovie(df)

In [11]:
def dictUpdate(genre, dfx, score, userClusters, genreModels):
    if genre == 'None':
        return
    
    model = genreModels[genre]
    clusterPredict = model.predict(dfx[['avg_vote','metascore']])
    cluster = int(clusterPredict)
    
    if genre in userClusters:
        if cluster in userClusters[genre]:
            scoreCluster = int(userClusters[genre][cluster][0] )
            numSeen = int(userClusters[genre][cluster][1])

            newScore = ((scoreCluster*numSeen) + score)/(numSeen + 1)
            newScore = int(newScore)

            userClusters[genre][cluster] = [newScore,numSeen + 1]

        else:
            userClusters[genre][cluster] = [score,1]
    else:
        userClusters[genre] = {}
        userClusters[genre][cluster] = [score,1]

In [12]:
# clustDict/userClusterScore= {genre:{clusterNo:[score(calculated),numberseen]}}
# movieClusters = {genre:{clusterLabel:[movies]}}
# movies - movietitle-director-year
def classify(dfx,utitle,score,clustDict,genreModels):
    g1 = list(dfx['genre 1'])[0]
    g2 = list(dfx['genre 2'])[0]
    g3 = list(dfx['genre 3'])[0]
    
    
    dictUpdate(g1, dfx, score, clustDict, genreModels)
    dictUpdate(g2, dfx, score, clustDict, genreModels)
    dictUpdate(g3, dfx, score, clustDict, genreModels)
    
    pass

In [13]:
def writtenReview(rev):
    score = 0
    
    sentimentModel = SentimentIntensityAnalyzer()
    greatWords = ['great','loved','fantastic','amazing','best']
    goodWords = ['nice','superb','fabulous','tremendous','mindnlowing','super','liked']
    badWords = ['terrible','bad','stupid','hated','hated']
    
    sentances = rev.split('.')
    gscore = 0
    bscore = 0
    nscore = 0
    
    prev = 'pos'
    for sentence in sentances:
        for words in sentence.split():
            if words in greatWords:
                gscore += 0.6
                nscore -= 0.4
                prev = 'pos'
                
            if words in goodWords:
                gscore += 0.25
                nscore -= 0.3
                prev = 'pos'
                
            if words in badWords:
                bscore += 0.5
                nscore -= 0.2
                prev = 'neg'
        sentscore = sentimentModel.polarity_scores(sentence)
        gscore += sentscore['pos']
        bscore += sentscore['neg']
        nscore += sentscore['neu']
#         print(sentence)
#         print(nscore,bscore,gscore)
        
        if sentscore['neu'] > sentscore['pos'] and sentscore['neu'] > sentscore['neg']:
            if prev == 'pos':
                gscore += sentscore['neu']
            else:
                bscore += sentscore['neu']
    
#     print('Final')
#     print(nscore,bscore,gscore)
    if nscore > gscore and nscore > bscore:
        score = 65
    
    else:
        score = (gscore - nscore)*100/len(sentances) 
        if score > 100:
            score = 100
        elif score < 0:
            score = 1
            
    return int(score)

# re = 'I loved this movie. it was the amazing. the acting was great. The effects could have been better'

# rev = writtenReview(re)
# print('Calculated score: ')
# print(rev)

In [119]:
greatWords = ['great','loved','fantastic','amazing','best']
'loved' in greatWords

True

In [16]:
user, userClusterScore = login(users)
while(1):
    print('1. Get recommendations')
    print('2. Review Movie')
    print('3. Switch user')
    print('4. User Profile')
    print('5. Multi User recommendation')
    print('x. Exit')
    print()
    ch = input()
    
    if ch == '1':
        print('\n Here are some suggestions for you')
        recommended = makeRecommendations(df, user, userClusterScore, similarityDictionary)
        for i in range(len(recommended)):
            print((i+1),'.',recommended[i])
        
        print('\n Select movie(0 to exit):')
        sel = int(input())
        if sel!= 0:
            print('1. Review ', recommended[sel-1])
            print('2. Mark ',recommended[sel-1],'as not interested')
            sel2 = (input())
            
            if sel2 == '1':
                ch = 'selectedReview'
                uTitle = recommended[sel-1]
                name,director,year = uTitle.split('|')
                mdf = df[df['title'] == name]
                mdf = mdf[mdf['director'] == director]
                
            else:
                user[recommended[sel-1]] = 50
    
    elif ch == '2':
        mdf = locateMovie(df)
        
        review = input('Enter movie review or just enter score(1-100):')
        if review.isnumeric():
            score = int(review)
        else:
            score = writtenReview(review)
        
        name = list(mdf['title'])[0]
        director = list(mdf['director'])[0]
        year = str(list(mdf['year'])[0])
        uTitle = name + '-' + director + '-' + year
        
        user[uTitle] = score
        
        classify(mdf, uTitle, score,userClusterScore, genreModel)
        # print(userClusterScore)
    
    elif ch == '3':
        login(users)
    
    elif ch == '4':
        print('1. Change movie length Preference')
        print('2. See Reviews')
        
        ch2 = input()
        if ch2 == '1':
            x = input('Minimum run time for movie:')
            y = input('Max run time for movie(0 for no limit):')
            print('Updated!')
    
    elif ch == '5':
        input('Enter other user names separated by commas: ')
        print('\n Here are some suggestions for you')
        recommended = makeRecommendations(df, user, userClusterScore, similarityDictionary)
        for i in range(len(recommended)):
            print((i+1),'.',recommended[i])
    
    elif ch == 'x':
        break
    
    else:
        print()
    
    #internal choice
    if ch == 'selectedReview':
        review = input('Enter movie review or just enter score(1-100):')
        if review.isnumeric():
            score = int(review)
        else:
            score = writtenReview(review)
        
        classify(mdf, uTitle, rating, userClusterScore, genreModel)
    print()

Enter User Name: user 1
Welcome back  user 1
1. Get recommendations
2. Review Movie
3. Switch user
4. User Profile
5. Multi User recommendation
x. Exit

5
Enter other user names separated by commas: user 2

 Here are some suggestions for you
1 . Hop|Tim Hill|2011
2 . Zootopia|Byron Howard, Rich Moore|2016
3 . Casino Royale|Martin Campbell|2006
4 . District 9|Neill Blomkamp|2009
5 . Brick|Rian Johnson|2005
6 . Life of Pi|Ang Lee|2012
7 . The Jungle Book 2|Steve Trenbirth|2003
8 . Aliens|James Cameron|1986
9 . Kona fer í stríð|Benedikt Erlingsson|2018
10 . Batman|Tim Burton|1989

1. Get recommendations
2. Review Movie
3. Switch user
4. User Profile
5. Multi User recommendation
x. Exit

x


In [14]:
users = {'user 1': [{'The Jungle Book-Jon Favreau-2016': 80,
   'The Jungle Book 2-Steve Trenbirth-2003': 63,
   'Iron Man-Jon Favreau-2008': 92,
   'Spider-Man-Sam Raimi-2002': 85,
   'Captain America: The Winter Soldier-Anthony Russo, Joe Russo-2014': 91,
   'Star Trek-J.J. Abrams-2009': 88,
   'Star Wars-George Lucas-1977': 76,
   'Star Trek II: The Wrath of Khan-Nicholas Meyer-1982': 91},
  {'Animation': {36: [63, 1], 3: [91, 1]},
   'Adventure': {3: [63, 1],
    122: [92, 1],
    58: [85, 1],
    29: [91, 2],
    50: [88, 1],
    40: [76, 1],
    148: [91, 1]},
   'Family': {53: [63, 1]},
   'Action': {154: [92, 1],
    192: [85, 1],
    53: [91, 1],
    9: [89, 2],
    139: [76, 1],
    70: [91, 1]},
   'Sci-Fi': {6: [90, 2], 30: [85, 1], 23: [91, 2]},
   'Fantasy': {45: [76, 1]}}]}

In [14]:
dnew = df[df['year']>2010]
dnew = dnew[dnew['year']<2017]
dnew = dnew[dnew['country']=='USA']
dnew = dnew[dnew['avg_vote']>70]
dnew

,title,year,genre,duration,country,director,writer,production_company,actors,avg_vote,metascore,genre 1,genre 2,genre 3
5772,42,2013,"Biography, Drama, Sport",128,USA,Brian Helgeland,Brian Helgeland,Warner Bros.,"Chadwick Boseman, Harrison Ford, Nicole Behari...",75.0,62.0,Biography,Drama,Sport
5800,The Equalizer,2014,"Action, Crime, Thriller",132,USA,Antoine Fuqua,"Richard Wenk, Michael Sloan",Columbia Pictures,"Denzel Washington, Marton Csokas, Chloë Grace ...",72.0,57.0,Action,Crime,Thriller
6018,Ant-Man,2015,"Action, Adventure, Comedy",117,USA,Peyton Reed,"Edgar Wright, Joe Cornish",Gary Sanchez Productions,"Paul Rudd, Michael Douglas, Evangeline Lilly, ...",73.0,64.0,Action,Adventure,Comedy
6046,The Croods,2013,"Animation, Action, Adventure",98,USA,"Kirk DeMicco, Chris Sanders","Chris Sanders, Kirk DeMicco",DreamWorks Animation,"Nicolas Cage, Emma Stone, Ryan Reynolds, Cathe...",72.0,55.0,Animation,Action,Adventure
6243,Drive,2011,"Crime, Drama",100,USA,Nicolas Winding Refn,"Hossein Amini, James Sallis",FilmDistrict,"Ryan Gosling, Carey Mulligan, Bryan Cranston, ...",78.0,78.0,Crime,Drama,None
6279,Dallas Buyers Club,2013,"Biography, Drama",117,USA,Jean-Marc Vallée,"Craig Borten, Melisa Wallack",Truth Entertainment (II),"Matthew McConaughey, Jennifer Garner, Jared Le...",80.0,84.0,Biography,Drama,None
6447,Hotel Transylvania,2012,"Animation, Comedy, Family",91,USA,Genndy Tartakovsky,"Peter Baynham, Robert Smigel",Columbia Pictures,"Adam Sandler, Andy Samberg, Selena Gomez, Kevi...",71.0,47.0,Animation,Comedy,Family
6486,The Avengers,2012,"Action, Adventure, Sci-Fi",143,USA,Joss Whedon,"Joss Whedon, Zak Penn",Marvel Studios,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",80.0,69.0,Action,Adventure,Sci-Fi
6600,Love & Mercy,2014,"Biography, Drama, Music",121,USA,Bill Pohlad,"Oren Moverman, Michael A. Lerner",River Road Entertainment,"Paul Dano, John Cusack, Elizabeth Banks, Paul ...",74.0,80.0,Biography,Drama,Music
6752,The Wolf of Wall Street,2013,"Biography, Crime, Drama",180,USA,Martin Scorsese,"Terence Winter, Jordan Belfort",Red Granite Pictures,"Leonardo DiCaprio, Jonah Hill, Margot Robbie, ...",82.0,75.0,Biography,Crime,Drama
